In [6]:
import os, pandas as pd, numpy as np
from datetime import datetime as dt

In [7]:
datapath = '../data/raw/jawiki/dumps_unzipped/'

In [8]:
mtdt = pd.read_csv('../references/metadata/history.tsv', sep='\t')
# print(mtdt.col_name.tolist())

### head

In [9]:
with open(datapath+'2021-12.jawiki.2013.tsv', 'r') as file:
    for i in range(3):
        print(file.readline())

jawiki	revision	create	2013-01-01 00:00:10.0	/* 関連項目 */		219.101.94.150	219.101.94.150										true						2414774	大間材木テレビ中継局	大間材木テレビ中継局	0	true	0	true	true	false	2011-08-23 15:00:28.0	2011-08-23 15:00:28.0	12	6771598																	45611822	44596655	false		false	3283	-1	h5njsxb8189dwoab2z5uh3wvsvn3dh0			false		false			false	false	

jawiki	revision	create	2013-01-01 00:00:17.0	/* 中継局概要 */		219.101.94.150	219.101.94.150										true						1185684	片貝中継局	片貝テレビ中継局	0	true	0	true	false	false	2007-10-08 03:33:18.0	2007-10-08 03:33:18.0	32	8974810																	45611823	44232137	false		false	3877	-16	hwp0qzjrb7a9qp46qbcii8qnk9aie2e			false		false			false	false	

jawiki	revision	create	2013-01-01 00:00:21.0			219.101.94.150	219.101.94.150										true						1202040	NHK三陸テレビ中継局	NHK三陸テレビ中継局	0	true	0	true	false	false	2007-10-21 01:19:51.0	2007-10-21 01:19:51.0	14	23793706																	45611824	41882385	false		false	2325	12	rc01e5nzqdiqret0snpjqeqfwghpb9l			false		false			false	false	



### split file so I can run csvsql to generate schema

#### copy 500 rows of the tsv

In [19]:
ct = 0
with open(datapath+'2021-12.jawiki.2013.tsv', 'r') as infile:
    with open('/home/bhrdwj/git/predwikt/data/processed/scratch_for_schema.tsv', 'w+') as outfile:
        while ct < 5*10**2:
            outfile.write(infile.readline())
            ct += 1
print(ct)

500


#### confirm the rows were copied

In [20]:
ct = 0
with open('/home/bhrdwj/git/predwikt/data/processed/scratch_for_schema.tsv', 'r') as file:
    for line in file:
        ct += 1
print(ct)

500


#### hand-edited query

In [12]:
make_history_table = """
CREATE TABLE ja_hist (	
wiki_db	VARCHAR(20) NOT NULL,
event_entity	VARCHAR(20) NOT NULL,
event_type	VARCHAR(20) NOT NULL,
event_timestamp	VARCHAR(30) NOT NULL,
event_comment	TEXT,
event_user_id	BIGINT,
event_user_text_historical	VARCHAR(80),
event_user_text	VARCHAR(80),
event_user_blocks_historical	VARCHAR(30),
event_user_blocks	VARCHAR(60),
event_user_groups_historical	VARCHAR(60),
event_user_groups	VARCHAR(200),
event_user_is_bot_by_historical	VARCHAR(10),
event_user_is_bot_by	VARCHAR(10),
event_user_is_created_by_self	BOOL,
event_user_is_created_by_system	BOOL,
event_user_is_created_by_peer	BOOL,
event_user_is_anonymous	BOOL,
event_user_registration_timestamp	VARCHAR(30),
event_user_creation_timestamp	VARCHAR(30),
event_user_first_edit_timestamp	VARCHAR(30),
event_user_revision_count	BIGINT,
event_user_seconds_since_previous_revision	BIGINT,
page_id	BIGINT,
page_title_historical	VARCHAR(200),
page_title	VARCHAR(200),
page_namespace_historical	INT,
page_namespace_is_content_historical	BOOL,
page_namespace	INT,
page_namespace_is_content	BOOL,
page_is_redirect	BOOL,
page_is_deleted	BOOL,
page_creation_timestamp	VARCHAR(30),
page_first_edit_timestamp	VARCHAR(30),
page_revision_count	BIGINT,
page_seconds_since_previous_revision	BIGINT,
user_id	BIGINT,
user_text_historical	VARCHAR(100),
user_text	VARCHAR(100),
user_blocks_historical	VARCHAR(50),
user_blocks	VARCHAR(50),
user_groups_historical	VARCHAR(50),
user_groups	VARCHAR(50),
user_is_bot_by_historical	VARCHAR(50),
user_is_bot_by	VARCHAR(50),
user_is_created_by_self	BOOL,
user_is_created_by_system	BOOL,
user_is_created_by_peer	BOOL,
user_is_anonymous	BOOL,
user_registration_timestamp	VARCHAR(30),
user_creation_timestamp	VARCHAR(30),
user_first_edit_timestamp	VARCHAR(30),
revision_id	BIGINT,
revision_parent_id	BIGINT,
revision_minor_edit	BOOL,
revision_deleted_parts	VARCHAR(30),
revision_deleted_parts_are_suppressed	BOOL,
revision_text_bytes	BIGINT,
revision_text_bytes_diff	BIGINT,
revision_text_sha1	VARCHAR(50),
revision_content_model	BOOL,
revision_content_format	BOOL,
revision_is_deleted_by_page_deletion	BOOL,
revision_deleted_by_page_deletion_timestamp	VARCHAR(30),
revision_is_identity_reverted	BOOL,
revision_first_identity_reverting_revision_id	BIGINT,
revision_seconds_to_identity_revert	BIGINT,
revision_is_identity_revert	BOOL,
revision_is_from_before_page_creation	BOOL,
revision_tags	VARCHAR(50),
row_id   BIGINT NOT NULL AUTO_INCREMENT,
PRIMARY KEY (row_id)
);
"""

### create table ja_hist

#### imports, login

In [13]:
import mysql.connector as mysql
import sqlalchemy

```bash
mysql --user=root --password=XXXXXXXX
```

```sql
CREATE DATABASE ja_hist CHARACTER SET utf8 COLLATE utf8_bin;
USE jawiki;
GRANT ALL PRIVILEGES ON ja_hist TO bhrdwj@localhost IDENTIFIED BY XXXXXXX;
GRANT ALL PRIVILEGES ON ja_hist.* TO bhrdwj@localhost IDENTIFIED BY XXXXXXX;
EXIT
```

In [4]:
mysql_superuser = 'root'
# mysql_su_pass = input(f'Enter the MySQL password for user {mysql_superuser}: ')

#### connect to database

In [21]:
host='localhost'; user=mysql_superuser; passwd=mysql_su_pass; dbname='ja_hist';
cxn = mysql.connect(host=host,user=user,passwd=passwd, database=dbname)
cur = cxn.cursor()

connection_str = 'mysql+mysqlconnector://'+user+':'+passwd+'@'+host+'/'+dbname  # removed this after host +':'+dbport
try:
    engine1 = sqlalchemy.create_engine(connection_str)
    conn1 = engine1.connect()
except:
    print('Database connection error - check creds')
metadata = sqlalchemy.MetaData(conn1)
metadata.reflect()
metadata.tables.keys()

dict_keys(['ja_hist'])

#### create table with schema

In [22]:
conn1.execute('DROP TABLE IF EXISTS ja_hist;')
conn1.execute(make_history_table)

In [26]:
a = conn1.execute('describe ja_hist;')
a.all()

[('wiki_db', 'varchar(20)', 'NO', '', None, ''),
 ('event_entity', 'varchar(20)', 'NO', '', None, ''),
 ('event_type', 'varchar(20)', 'NO', '', None, ''),
 ('event_timestamp', 'varchar(30)', 'NO', '', None, ''),
 ('event_comment', 'text', 'YES', '', None, ''),
 ('event_user_id', 'bigint(20)', 'YES', '', None, ''),
 ('event_user_text_historical', 'varchar(80)', 'YES', '', None, ''),
 ('event_user_text', 'varchar(80)', 'YES', '', None, ''),
 ('event_user_blocks_historical', 'varchar(30)', 'YES', '', None, ''),
 ('event_user_blocks', 'varchar(60)', 'YES', '', None, ''),
 ('event_user_groups_historical', 'varchar(60)', 'YES', '', None, ''),
 ('event_user_groups', 'varchar(200)', 'YES', '', None, ''),
 ('event_user_is_bot_by_historical', 'varchar(10)', 'YES', '', None, ''),
 ('event_user_is_bot_by', 'varchar(10)', 'YES', '', None, ''),
 ('event_user_is_created_by_self', 'tinyint(1)', 'YES', '', None, ''),
 ('event_user_is_created_by_system', 'tinyint(1)', 'YES', '', None, ''),
 ('event_user

### insert data

Reference: ([link](https://mariadb.com/kb/en/load-data-infile/))  
```sql
LOAD DATA INFILE 'ld.txt' INTO TABLE ld (@i,v) SET i=@i*2;
```

In [24]:
load_data_sql = """
LOAD DATA LOCAL INFILE '/home/bhrdwj/git/predwikt/data/processed/scratch_for_schema.tsv'
INTO TABLE ja_hist 
CHARACTER SET utf8mb4
FIELDS TERMINATED BY "\t";
"""

In [25]:
conn1.execute(load_data_sql)

START 19:24PM

In [27]:
tblname='ja_hist'
sql_eda2 = f'select count(*) from {tblname};'
print(dt.now())
sql_rows = pd.read_sql(sql_eda2,engine1)
print(sql_rows)
print(dt.now())

2022-01-07 09:56:18.167770
   count(*)
0         0
2022-01-07 09:56:18.185921
